# Python Imports

Code and errors written by Thomas Martin 

In [1]:
import pandas as pd
import numpy as np
import random

from tqdm import tqdm
import glob

### File Paths:

In [2]:
import_path = '/glade/p/cisl/aiml/ai2es/winter_ptypes/mping_hourly/'

In [3]:
export_path = '/glade/p/cisl/aiml/ai2es/winter_ptypes/mping_hourly_pq/'

Getting a list of JSON files:

In [4]:
json_files = glob.glob(import_path + "*.json")

In [5]:
len(json_files)

83232

## Converting JSON files into dataframes

In [6]:
%%time

dfs = []

for file in tqdm(json_files):
    try:
        if not file.endswith('.json'):
            raise TypeError(f"File '{file}' is not a JSON file.")
        df = pd.read_json(file)
        dfs.append(df)
    except TypeError as te:
        print(te)
    except Exception as e:
        print(f"Error reading file '{file}': {str(e)}")

100%|██████████| 83232/83232 [03:17<00:00, 422.34it/s]

CPU times: user 3min 8s, sys: 6.55 s, total: 3min 15s
Wall time: 3min 17s


In [7]:
combined_df = pd.concat(dfs, ignore_index=True)

In [8]:
mem = combined_df.memory_usage()
type(mem)

pandas.core.series.Series

This math right?

In [9]:
print('Megabytes of dataframe', mem.sum() / 1000000 )

Megabytes of dataframe 109.83704


## Precip Description

In [10]:
combined_df.shape

(2288269, 6)

In [11]:
combined_df.description.unique()

array(['Snow and/or Graupel', 'Rain', 'Ice Pellets/Sleet',
       'Freezing Drizzle', 'Mixed Ice Pellets and Snow', 'Freezing Rain',
       'Drizzle', 'Mixed Freezing Rain and Ice Pellets',
       'Mixed Rain and Snow', 'Mixed Rain and Ice Pellets', 'Graupel'],
      dtype=object)

In [12]:
my_dict = {
    'value_to_match': ['Freezing Drizzle', 'Ice Pellets', 'Mixed Ice Pellets and Snow', 'Mixed Freezing Rain and Ice Pellets', 'Mixed Rain and Snow', 'Mixed Rain and Ice Pellets', 'Drizzle', 'Graupel', 'Ice Pellets/Sleet', 'Snow and/or Graupel'],
    'new_value':      ['Freezing Rain', 'Freezing Rain', 'Snow', 'Ice Pellets', 'Snow', 'Ice Pellets', 'Rain', 'Snow', 'Freezing Rain', 'Snow']
}

column_to_match = 'description'

In [13]:
combined_df['duplicate'] = False

In [14]:
for num in range(len(my_dict['value_to_match'])):
    print(my_dict['value_to_match'][num])
    row_to_copy = combined_df.loc[combined_df[column_to_match] == my_dict['value_to_match'][num]].copy()
    print('dataframe shape:', row_to_copy.shape)
    print('changed to:')
    print(my_dict['new_value'][num])
    row_to_copy[column_to_match] = my_dict['new_value'][num]
    row_to_copy['duplicate'] = True
    
    combined_df = pd.concat([combined_df, row_to_copy]) # this appends that dataframe to the original
    print('   ')

Freezing Drizzle
dataframe shape: (27016, 7)
changed to:
Freezing Rain
   
Ice Pellets
dataframe shape: (0, 7)
changed to:
Freezing Rain
   
Mixed Ice Pellets and Snow
dataframe shape: (35831, 7)
changed to:
Snow
   
Mixed Freezing Rain and Ice Pellets
dataframe shape: (9420, 7)
changed to:
Ice Pellets
   
Mixed Rain and Snow
dataframe shape: (66520, 7)
changed to:
Snow
   
Mixed Rain and Ice Pellets
dataframe shape: (31960, 7)
changed to:
Ice Pellets
   
Drizzle
dataframe shape: (245493, 7)
changed to:
Rain
   
Graupel
dataframe shape: (3959, 7)
changed to:
Snow
   
Ice Pellets/Sleet
dataframe shape: (101276, 7)
changed to:
Freezing Rain
   
Snow and/or Graupel
dataframe shape: (624566, 7)
changed to:
Snow
   


In [15]:
combined_df[combined_df.duplicate == True].description.unique()

array(['Freezing Rain', 'Snow', 'Ice Pellets', 'Rain'], dtype=object)

In [16]:
combined_df[combined_df.duplicate == False].description.unique()

array(['Snow and/or Graupel', 'Rain', 'Ice Pellets/Sleet',
       'Freezing Drizzle', 'Mixed Ice Pellets and Snow', 'Freezing Rain',
       'Drizzle', 'Mixed Freezing Rain and Ice Pellets',
       'Mixed Rain and Snow', 'Mixed Rain and Ice Pellets', 'Graupel'],
      dtype=object)

In [17]:
combined_df.shape

(3434310, 7)

This would be the place to adjust/fix your precip choices.

## Coordinate Extraction

In [18]:
combined_df['coordinates'] = combined_df['geom'].apply(lambda x: x['coordinates'])

In [19]:
combined_df

,id,obtime,category,description,description_id,geom,duplicate,coordinates
0,1128052,2016-11-21T18:05:08Z,Rain/Snow,Snow and/or Graupel,8,"{'type': 'Point', 'coordinates': [-72.53477752...",False,"[-72.53477752710911, 41.78177793818802]"
1,1128055,2016-11-21T18:14:57Z,Rain/Snow,Snow and/or Graupel,8,"{'type': 'Point', 'coordinates': [-74.0841645,...",False,"[-74.0841645, 40.9787401]"
2,1128056,2016-11-21T18:18:31Z,Rain/Snow,Snow and/or Graupel,8,"{'type': 'Point', 'coordinates': [-77.63962137...",False,"[-77.63962137520093, 43.2546242745996]"
3,1128057,2016-11-21T18:25:43Z,Rain/Snow,Snow and/or Graupel,8,"{'type': 'Point', 'coordinates': [-72.6836217,...",False,"[-72.6836217, 41.7416863]"
4,1128058,2016-11-21T18:29:49Z,Rain/Snow,Snow and/or Graupel,8,"{'type': 'Point', 'coordinates': [-73.9366576,...",False,"[-73.9366576, 41.6538321]"
...,...,...,...,...,...,...,...,...
2288257,2904383,2021-12-04T18:22:10Z,Rain/Snow,Snow,8,"{'type': 'Point', 'coordinates': [-78.9275127,...",True,"[-78.9275127, 43.8519138]"
2288259,2904385,2021-12-04T18:25:13Z,Rain/Snow,Snow,8,"{'type': 'Point', 'coordinates': [-73.70230064...",True,"[-73.70230064948555, 44.01912704475374]"
2288263,2904389,2021-12-04T18:43:50Z,Rain/Snow,Snow,8,"{'type': 'Point', 'coordinates': [-108.7052384...",True,"[-108.70523847448939, 47.67775311387512]"
2288266,2904392,2021-12-04T18:49:30Z,Rain/Snow,Snow,8,"{'type': 'Point', 'coordinates': [-121.8419645...",True,"[-121.8419645329683, 48.10779571646841]"


In [20]:
# Split coordinates into separate columns
combined_df['latitude'] = combined_df['coordinates'].str[1]
combined_df['longitude'] = combined_df['coordinates'].str[0]

remove three columns

In [21]:
combined_df = combined_df.drop(columns=['geom', 'coordinates', 'description_id' ])

## Round down time to nearest hour

In [22]:
combined_df['date'] = pd.to_datetime(combined_df['obtime' ])

In [23]:
combined_df['rounded_datetime'] = combined_df['date'].dt.floor('H')

#dropping obtime
combined_df = combined_df.drop(columns=['obtime'])

## Pandas EDA

In [24]:
print(combined_df.dtypes)

id                                int64
category                         object
description                      object
duplicate                          bool
latitude                        float64
longitude                       float64
date                datetime64[ns, UTC]
rounded_datetime    datetime64[ns, UTC]
dtype: object


## Export Every day

In [25]:
# Group the dataframe by day
grouped = combined_df.groupby(combined_df['date'].dt.date)

In [26]:
# Iterate over the groups and export each day as a separate file
for date, group in tqdm(grouped, desc='Exporting data', unit='day'):
    filename = f'data_{date}.parquet'  # Define the filename for each day
    group.to_parquet(export_path + filename, index=False)  # Export the group to a Parquet file

Exporting data: 100%|██████████| 3427/3427 [00:21<00:00, 160.14day/s]


## Double check that each day is actually it's own day:

In [27]:
pq_files = glob.glob(export_path + "*.parquet")

In [28]:
random_files = random.sample(pq_files, 5)

In [29]:
for file in random_files:
    # Read the file using pd.read_parquet (for Parquet files)
    df = pd.read_parquet(file)
    # Display the first 3 rows of the DataFrame
    print(file)
    print(df.date.head(2))
    print(df.date.tail(2))

/glade/p/cisl/aiml/ai2es/winter_ptypes/mping_hourly_pq/data_2020-01-19.parquet
0   2020-01-19 10:06:45+00:00
1   2020-01-19 10:21:40+00:00
Name: date, dtype: datetime64[ns, UTC]
2418   2020-01-19 00:58:52+00:00
2419   2020-01-19 00:59:21+00:00
Name: date, dtype: datetime64[ns, UTC]
/glade/p/cisl/aiml/ai2es/winter_ptypes/mping_hourly_pq/data_2017-01-15.parquet
0   2017-01-15 10:58:52+00:00
1   2017-01-15 12:04:56+00:00
Name: date, dtype: datetime64[ns, UTC]
873   2017-01-15 15:45:00+00:00
874   2017-01-15 15:50:26+00:00
Name: date, dtype: datetime64[ns, UTC]
/glade/p/cisl/aiml/ai2es/winter_ptypes/mping_hourly_pq/data_2018-11-09.parquet
0   2018-11-09 14:00:27+00:00
1   2018-11-09 14:02:04+00:00
Name: date, dtype: datetime64[ns, UTC]
2661   2018-11-09 04:53:45+00:00
2662   2018-11-09 04:58:14+00:00
Name: date, dtype: datetime64[ns, UTC]
/glade/p/cisl/aiml/ai2es/winter_ptypes/mping_hourly_pq/data_2017-10-24.parquet
0   2017-10-24 01:02:54+00:00
1   2017-10-24 01:04:56+00:00
Name: date, dt